In [ ]:
import re
from pathlib import Path
import pandas as pd
import pdb

#Cle

In [ ]:
data_folder = Path("/Users/nicolasroever/Dropbox/Promotion/LVT/landvaluetax/src/landvaluetax/data/raw_data_res_land_quarter")

In [ ]:
test = pd.read_excel("/Users/nicolasroever/Dropbox/Promotion/LVT/landvaluetax/src/landvaluetax/data/raw_data_res_land_quarter/Real property price statistics (34).xlsx")

In [ ]:
all_dfs = []

In [ ]:
i = 0
for file_path in data_folder.glob("*.xlsx"):
    i += 1
    print(f"Processing file {i}: {file_path.name}")


    # --- a) Read the transaction_type (first row, first cell) ---
    transaction_type = pd.read_excel(file_path, header=None, nrows=1).iloc[0, 0]

    # --- b) Read the info string (second row, first cell) ---
    info_str = pd.read_excel(file_path, header=None, skiprows=1, nrows=1).iloc[0, 0]

    parts = info_str.split(",", 1)
    county, municipality = parts[0].strip(), parts[1].strip()

    # --- c) Read the data table starting at row 7 (skip 6 rows) ---
    raw = pd.read_excel(file_path, header=None, skiprows=6)
    # drop everything at and after the first blank row
    blank_idx = raw[raw.isna().all(axis=1)].index
    if len(blank_idx):
        raw = raw.loc[: blank_idx[0] - 1]

    # assign your column names
    raw.columns = [
        'year_quarter',
        'house_type',
        "number_trans",
        "average_sqm",
        "total_price",
        "min_price",
        "max_price",
        "min_price_per_sqm",
        "max_price_per_sqm",
        "median_price_per_sqm",
        "average_price_per_sqm",
        "sd_price_per_sqm",
    ]

    # --- d) Add the metadata columns ---
    raw["county"] = county
    raw["municipality"] = municipality


    # collect
    all_dfs.append(raw)

# 4) Concatenate all into one big DataFrame
result = pd.concat(all_dfs, ignore_index=True)

In [ ]:
result.head()

In [ ]:
result.to_csv("/Users/nicolasroever/Dropbox/Promotion/LVT/landvaluetax/bld/data/quarterly_prices_res_land.csv", index=False)